# Parallel Feature Engineering: Comparing Sequential and Multiprocessing Approaches

Created by Gaurav Kaushik

In this notebook, we explore how to speed up feature engineering by using **parallel processing**.

We fabricate a **large version** of the California Housing dataset (4 million+ rows) to simulate a real-world big data environment.

We will:
- Create 6 new engineered features
- Apply feature engineering sequentially
- Apply feature engineering using multiprocessing
- Compare the time taken by both approaches

This demonstrates how **multiprocessing** can scale feature engineering for real data science tasks.


## 1. Overview

Feature engineering is one of the most important steps in building machine learning models.

As datasets grow larger, sequential feature engineering becomes slow and inefficient.

In this notebook, I will show:
- How to build features sequentially
- How to parallelize feature generation
- How much time we can save using multiprocessing


## 2. Problem Setup

- **Dataset**: Large fabricated version of California Housing dataset
- **Size**: 4,128,000 rows and 9 columns
- **Task**: Create 6 new features
- **Goal**: Compare sequential and parallel feature engineering performance


Let's load the dataset.

In [1]:
import pandas as pd

# Load the large fabricated dataset
df_large = pd.read_csv("housing_large.csv")

# Check the shape
print(f"Dataset shape: {df_large.shape}")
df_large.head()

Dataset shape: (4128000, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## 3. Features to Create

We will engineer the following features:

| Feature Name         | Description |
|----------------------|-------------|
| RoomsPerPerson        | Average number of rooms per person (`AveRooms / AveOccup`) |
| BedroomsPerRoom       | Average number of bedrooms per room (`AveBedrms / AveRooms`) |
| PopulationDensity     | Population divided by number of households (`Population / AveOccup`) |
| IsHighIncome          | 1 if `MedInc` > dataset median, else 0 |
| AgeBucket             | Categorize `HouseAge` into Young, Mid, Old |
| Coordinates           | Combine `Latitude` and `Longitude` as a tuple


## 4. Approach 1: Sequential Feature Engineering

- Create all 6 features one-by-one.
- No parallelism involved.
- Measure and record total time taken.


Let's create features sequentially.


In [3]:
import time

# Make a fresh copy of dataset.
df_seq = df_large.copy()

# Start timing
start_time = time.time()

# Feature 1: Rooms per Person
df_seq["RoomsPerPerson"] = df_seq["AveRooms"] / df_seq["AveOccup"]

# Feature 2: Bedrooms per Room
df_seq["BedroomsPerRoom"] = df_seq["AveBedrms"] / df_seq["AveRooms"]

# Feature 3: Population Density
df_seq["PopulationDensity"] = df_seq["Population"] / df_seq["AveOccup"]

# Feature 4: High Income Flag
median_income = df_seq["MedInc"].median()
df_seq["IsHighIncome"] = (df_seq["MedInc"] > median_income).astype(int)

# Feature 5: Age Bucket
def bucketize_age(age):
    if age < 20:
        return "Young"
    elif 20 <= age <= 40:
        return "Mid"
    else:
        return "Old"
    
df_seq["AgeBucket"] = df_seq["HouseAge"].apply(bucketize_age)

# Feature 6: Coordinates as Tuple
df_seq["Coordinates"] = list(zip(df_seq["Latitude"], df_seq["Longitude"]))

# End timing
end_time = time.time()

print(f"Sequential feature engineering time: {round(end_time - start_time, 2)} seconds")

Sequential feature engineering time: 3.76 seconds


## 5. Approach 2: Parallel Feature Engineering (Multiprocessing)

Instead of creating features one-by-one sequentially, we now:

- Define each feature creation as an independent function.
- Use Python's `multiprocessing.Pool` to run multiple functions in parallel.
- Merge the results back into a single dataframe.
- Measure and record total time taken.

---
### 🔄 Why Not Multithreading? Why Multiprocessing?

Before choosing how to parallelize our feature engineering, we considered:

#### ❓ Are the tasks I/O-bound or CPU-bound?
- **I/O-bound** tasks involve waiting (e.g., reading files, calling APIs). These benefit from **multithreading**.
- **CPU-bound** tasks involve heavy computation (e.g., math, transformation). These benefit from **multiprocessing**.

#### 💥 What’s the issue with threads in Python?

Python has something called the **Global Interpreter Lock (GIL)**.

> 🔒 The GIL allows **only one thread to execute Python code at a time**, even on multi-core machines.

So even if you use `threading`, only **one thread runs Python bytecode at a time**. This means:
- No real parallelism for CPU-bound tasks
- Threads end up waiting on each other

That’s why multithreading doesn’t speed up heavy computations.

---

#### ✅ Why Multiprocessing Works

Multiprocessing creates **separate processes**, each with:
- Its own Python interpreter
- Its own memory space
- Full access to a CPU core

This bypasses the GIL and allows true parallel execution on multi-core systems — ideal for our math-heavy feature generation.

✅ So we chose **multiprocessing** because our tasks are:
- Independent
- CPU-intensive
- Suitable for parallel processing across multiple cores

Let's write the code


In [4]:
# Define each feature function.

# Define independent feature functions
def add_rooms_per_person(df):
    df["RoomsPerPerson"] = df["AveRooms"] / df["AveOccup"]
    return df[["RoomsPerPerson"]]

def add_bedrooms_per_room(df):
    df["BedroomsPerRoom"] = df["AveBedrms"] / df["AveRooms"]
    return df[["BedroomsPerRoom"]]

def add_population_density(df):
    df["PopulationDensity"] = df["Population"] / df["AveOccup"]
    return df[["PopulationDensity"]]

def add_high_income_flag(df):
    median_income = df["MedInc"].median()
    df["IsHighIncome"] = (df["MedInc"] > median_income).astype(int)
    return df[["IsHighIncome"]]

def add_age_bucket(df):
    def bucketize(age):
        if age < 20:
            return "Young"
        elif 20 <= age <= 40:
            return "Mid"
        else:
            return "Old"
    df["AgeBucket"] = df["HouseAge"].apply(bucketize)
    return df[["AgeBucket"]]

def add_coordinates(df):
    df["Coordinates"] = list(zip(df["Latitude"], df["Longitude"]))
    return df[["Coordinates"]]


In [ ]:
from multiprocessing import Pool, cpu_count
from functools import reduce

# wrapper function

def apply_feature_func(func):
    func(df_large.copy())

feature_functions = [
    add_rooms_per_person,
    add_bedrooms_per_room,
    add_population_density,
    add_high_income_flag,
    add_age_bucket,
    add_coordinates
]


start_time = time.time()

    # Create a Pool and apply functions
with Pool(processes=cpu_count()) as pool:
    feature_dfs = pool.map(apply_feature_func, feature_functions)

# Merge features back into one dataframe
df_parallel = pd.concat(feature_dfs, axis=1)

# End timing
end_time = time.time()

print(f"Multiprocessing feature engineering time: {round(end_time - start_time, 2)} seconds")


## 6. Performance Comparison

| Method           | Time Taken |
|------------------|------------|
| Sequential       | X seconds  |
| Multiprocessing  | Y seconds  |

- Multiprocessing should show major speed improvements as dataset size increases.
- It reduces total processing time by utilizing multiple CPU cores.


## 7. Key Learnings

- **Multiprocessing** helps accelerate CPU-bound tasks like feature engineering.
- Ideal when:
  - Feature functions are independent
  - Dataset is large enough to offset parallelization overhead
- **Multithreading** is NOT effective for CPU-bound tasks in Python (due to GIL).
- Always measure the time gain when switching to parallel approaches.
